In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import sqlite3

In [35]:
teams = {
  "Hawks": "ATL",
  "Celtics": "BOS",
  "Nets": "BKN",
  "Hornets" : "CHA",
  "Bobcats" : "CHA",
  "Bulls" : "CHI",
  "Cavaliers" : "CLE",
  "Mavericks" : "DAL",
  "Nuggets" : "DEN",
  "Pistons" : "DET",
  "Warriors" : "GSW",
  "Rockets" : "HOU",
  "Pacers" : "IND",
  "Clippers" : "LAC",
  "Lakers" : "LAL",
  "Grizzlies" : "MEM",
  "Heat" : "MIA",
  "Bucks" : "MIL",
  "Timberwolves" : "MIN",
  "Pelicans" : "NOP",
  "Knicks" : "NYK",
  "Thunder" : "OKC",
  "Magic" : "ORL",
  "76ers" : "PHI",
  "Suns" : "PHX",
  "Blazers" : "POR",
  "Kings" : "SAC",
  "Spurs" : "SAS",
  "Raptors" : "TOR",
  "Jazz" : "UTA",
  "Wizards" : "WAS",
  "Bullets" : "WAS"
}

def get_nba_season_year(date):
    if date.month >= 7:
        return date.year + 1
    else:
        return date.year

In [36]:
injury_stats_df = pd.read_csv("NBA Player Injury Stats.csv")
injury_stats_df.drop(columns=['Unnamed: 0'], inplace=True)
injury_stats_df['Date'] = pd.to_datetime(injury_stats_df['Date'], errors='coerce')
injury_stats_df = injury_stats_df[injury_stats_df['Date'] > '2008-08-01']
injury_stats_df['Injured'] = pd.isna(injury_stats_df['Acquired'])
injury_stats_df['Acquired'] = injury_stats_df['Acquired'].str.strip()
injury_stats_df['Relinquished'] = injury_stats_df['Relinquished'].str.strip()
injury_stats_df['Notes'] = injury_stats_df['Notes'].str.strip()
injury_stats_df = injury_stats_df[injury_stats_df['Injured'] ^ (~pd.isna(injury_stats_df['Relinquished'])) == False]
injury_stats_df.loc[injury_stats_df['Injured'] == True, 'Player'] = injury_stats_df['Relinquished']
injury_stats_df.loc[injury_stats_df['Injured'] == False, 'Player'] = injury_stats_df['Acquired']
injury_stats_df = injury_stats_df.drop(columns=['Relinquished', 'Acquired'])
injury_stats_df['Team'] = [teams[i] for i in injury_stats_df['Team'].str.strip()]
injury_stats_df

,Date,Team,Notes,Injured,Player
12576,2008-10-28,POR,placed on IL with left foot injury,True,Martell Webster
12577,2008-10-28,POR,placed on IL with right shoulder injury,True,Raef LaFrentz
12578,2008-10-28,POR,placed on IL,True,Shavlik Randolph
12579,2008-10-28,MIL,placed on IL,True,Austin Croshere
12580,2008-10-28,MIL,placed on IL,True,Damon Jones
...,...,...,...,...,...
37662,2023-04-16,LAC,activated from IL,False,Marcus Morris
37663,2023-04-16,MEM,activated from IL,False,Dillon Brooks
37664,2023-04-16,MEM,activated from IL,False,Ja Morant
37665,2023-04-16,MEM,activated from IL,False,Jaren Jackson Jr.


In [37]:
with zipfile.ZipFile('basketball-final.sqlite.zip', 'r') as zip_ref:
    zip_ref.extractall()
conn = sqlite3.connect('basketball-final.sqlite')
betting_df = pd.read_sql_query("SELECT * FROM BettingOdds_History;", conn)
betting_df['Date'] = pd.to_datetime(betting_df['Date'])
betting_df = betting_df[betting_df['Date'] > '2008-08-01']
betting_df.loc[betting_df['HomeTeam'] == 'NJN', 'HomeTeam'] = 'BKN'
betting_df.loc[betting_df['HomeTeam'] == 'NOH', 'HomeTeam'] = 'NOP'
betting_df.loc[betting_df['AwayTeam'] == 'NJN', 'AwayTeam'] = 'BKN'
betting_df.loc[betting_df['AwayTeam'] == 'NOH', 'AwayTeam'] = 'NOP'
betting_df

,GAME_ID,Date,HomeTeam,AwayTeam,HomeSpread_AtOpen,HomeSpread_AtClose,Over_AtOpen,Over_AtClose,HomeML,AwayML,2H_HomeSpread,2H_Over
1316,0020800001,2008-10-28,BOS,CLE,-7.0,-6.0,179.5,182.5,-260,220,-5.0,93.5
1317,0020800002,2008-10-28,CHI,MIL,-6.0,-6.0,196.0,198.0,-270,230,-3.0,101.5
1318,0020800003,2008-10-28,LAL,POR,-7.0,-8.0,196.5,195.0,-370,305,1.0,96.0
1319,0020800005,2008-10-29,PHI,TOR,-5.0,-5.0,190.0,187.5,-210,175,-4.5,94.0
1320,0020800004,2008-10-29,ORL,ATL,-7.5,-8.0,202.0,202.5,-370,310,-8.5,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...
18287,0022100460,2021-12-20,MEM,OKC,-10.0,-9.0,211.5,215.5,-400,330,-5.5,105.0
18288,0022100459,2021-12-20,CHI,HOU,-8.5,-7.5,223.5,223.0,-280,240,1.5,107.0
18289,0022100461,2021-12-20,UTA,CHA,-11.0,-12.5,234.0,237.5,-900,600,-3.0,114.0
18290,0022100462,2021-12-20,GSW,SAC,-13.5,-13.5,223.5,221.5,-1000,650,-3.5,107.0


In [38]:
player_data = pd.read_csv("nba_player.csv")
player_data.rename(columns={"season": "Season", "player_name" : "Player"}, inplace=True)
player_data

,Player,Season,pos,age,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,...,usg_pct,ows,dws,ws_per_48,obpm,dbpm,bpm,vorp,mp,pk
0,A.C. Green,1995,SF,31,14.2,0.596,0.206,0.556,8.3,20.4,...,14.2,4.6,2.1,0.120,0.8,-0.9,-0.1,1.3,3055,23.0
1,Aaron McKie,1995,SG,22,13.3,0.500,0.107,0.280,4.6,13.5,...,17.0,0.8,1.2,0.115,-1.4,1.7,0.3,0.5,861,17.0
2,Aaron Swinson,1995,SF,24,11.7,0.594,0.000,0.278,6.7,11.3,...,21.4,0.0,0.0,0.039,-4.3,-2.5,-6.8,-0.1,51,61.0
3,Aaron Williams,1995,PF,23,7.0,0.410,0.042,0.500,8.1,24.1,...,22.3,-0.2,0.1,-0.067,-8.7,-0.3,-9.0,-0.1,72,61.0
4,Acie Earl,1995,C,24,6.6,0.409,0.000,0.426,10.4,14.6,...,19.7,-0.4,0.2,-0.038,-6.9,-2.0,-8.9,-0.4,218,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14191,Zach Collins,2023,C,25,16.4,0.599,0.268,0.259,8.5,22.5,...,21.2,1.2,1.0,0.074,-0.5,0.3,-0.2,0.6,1441,10.0
14192,Zach LaVine,2023,SG,27,19.0,0.607,0.392,0.308,1.7,12.2,...,28.3,4.2,2.9,0.123,2.7,-0.7,1.9,2.7,2768,13.0
14193,Zeke Nnaji,2023,PF,22,12.5,0.620,0.332,0.316,10.5,11.2,...,15.2,0.9,0.7,0.106,-2.7,-0.9,-3.7,-0.3,728,22.0
14194,Ziaire Williams,2023,SF,21,7.4,0.511,0.495,0.112,3.0,11.9,...,18.0,-0.5,0.6,0.011,-4.7,-0.8,-5.6,-0.5,561,10.0


In [39]:
betting_df['Team'] = betting_df['HomeTeam']
merged_df1 = pd.merge(betting_df, injury_stats_df, on=['Date', 'Team'], how='inner')
betting_df['Team'] = betting_df['AwayTeam']
merged_df2 = pd.merge(betting_df, injury_stats_df, on=['Date', 'Team'], how='inner')
merged_df = pd.concat([merged_df1, merged_df2], axis=0)
merged_df["Season"] = merged_df["Date"].apply(get_nba_season_year)
final_data = pd.merge(merged_df, player_data, on=['Player', 'Season'], how='inner')
all = pd.read_sql_query("SELECT GAME_ID, WL_HOME FROM Game;", conn)
final_data = pd.merge(final_data, all, on=['GAME_ID'], how='inner')
final_data['WL_HOME'] = final_data['WL_HOME'].map({'W': 1, 'L': 0})
final_data.dropna(how='any', inplace=True)
final_data = final_data.sort_values(by=['Season', 'Date', 'HomeTeam', 'AwayTeam', 'Team', 'Player'])
final_data

,GAME_ID,Date,HomeTeam,AwayTeam,HomeSpread_AtOpen,HomeSpread_AtClose,Over_AtOpen,Over_AtClose,HomeML,AwayML,...,ows,dws,ws_per_48,obpm,dbpm,bpm,vorp,mp,pk,WL_HOME
0,0020800001,2008-10-28,BOS,CLE,-7.0,-6.0,179.5,182.5,-260,220,...,0.000000,0.000000,0.201000,2.900000,4.200000,7.200000,0.000000,8,30.0,1
9319,0020800001,2008-10-28,BOS,CLE,-7.0,-6.0,179.5,182.5,-260,220,...,0.200000,0.700000,0.094000,-5.400000,0.200000,-5.200000,-0.400000,455,52.0,1
9320,0020800001,2008-10-28,BOS,CLE,-7.0,-6.0,179.5,182.5,-260,220,...,0.000000,0.000000,0.045000,-1.100000,-2.500000,-3.500000,0.000000,20,61.0,1
1,0020800002,2008-10-28,CHI,MIL,-6.0,-6.0,196.0,198.0,-270,230,...,0.607099,0.980325,0.069942,-0.898377,0.426775,-0.471602,0.457099,1479,8.0,1
2,0020800002,2008-10-28,CHI,MIL,-6.0,-6.0,196.0,198.0,-270,230,...,0.000000,0.100000,0.080000,-4.900000,1.900000,-3.000000,0.000000,40,32.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9318,0022100463,2021-12-20,LAC,SAS,-4.5,-6.0,216.5,220.0,-230,195,...,-0.600000,1.900000,0.057000,1.800000,1.900000,3.700000,1.600000,1077,10.0,0
17247,0022100463,2021-12-20,LAC,SAS,-4.5,-6.0,216.5,220.0,-230,195,...,1.100000,0.500000,0.062000,-0.400000,-2.100000,-2.500000,-0.100000,1223,11.0,0
9313,0022100460,2021-12-20,MEM,OKC,-10.0,-9.0,211.5,215.5,-400,330,...,4.600000,2.100000,0.171000,6.200000,-0.100000,6.100000,3.900000,2227,2.0,0
9312,0022100460,2021-12-20,MEM,OKC,-10.0,-9.0,211.5,215.5,-400,330,...,3.400000,1.600000,0.158000,1.200000,0.500000,1.700000,1.400000,1811,24.0,0


In [40]:
final_data.to_csv('data.csv', index=False)  